# Replica Set

A [ReplicaSet](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/)'s purpose is to maintain a stable set of replica Pods running at any given time.

ReplicaSet的目的是讓Pods副本在任何時間都能維持同一狀態

> 在minikube的環境下，access Cluster IP或是 access Pod IP的時候要用 minikube ssh, nodeIP 着不需要 minikubessh

In [ ]:
from IPython.display import Image
Image(url="./img/replicaset.jpg", width=300, height=300)

## 什麼時候用ReplicaSet

A ReplicaSet ensures that a sepecified nubmer of pod replicas are running at any given time, it is suitable for Stateless application.

ReplicaSet可以保持指定數量的Pods副本在任何時間繼續運行.

## 什麼是無狀態應用 Stateless

Stateless 是指這個應用服務或功能，不因任何狀態而改變回傳的資料。
- eg: 沒有資料庫的static web

## 什麼是有狀態應用  StateFul

Stateful 則與 Stateless 相反，會因狀態而改變回傳的資料，像是Request的次數。
- eg: 有資料庫的 wordpress

## Replica Set

而ReplicaSet 正適合與建立stateless服務，因爲Pod在消滅後再建立的話，是不保存之前的狀態的。

ReplicaSet:
- 管理一群Pods
- 透過 Selector 選擇那些Pod 需要進行管理
- 每個 ReplicaSet 可以根據需求建立或刪除Pod使得副本達到期望值
- 利用template建立Container


In [101]:
# 確保你的minikube有啟動
! [[ -z $(kubectl get nodes) ]] && minikube start 
# 列印目前k8s節點
! kubectl get nodes                    

NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   53m   v1.25.3


In [102]:
! cat replicaset.yaml             

apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: hostname
  labels:
    app: hostname
spec:
  # modify replicas according to your case
  replicas: 3
  selector:
    matchLabels:
      tier: frontend
  template:
    metadata:
      labels:
        tier: frontend
    spec:
      containers:
      - name: hostname
        image: docker.io/brokenpen/hostname:v1

- yaml
  - apiVersion
  - kind
  - metadata
  - spec
     - replicas # 指定要維持多少分replica
     - selector # 
     - template # container template

In [103]:
# 透過yaml 建立 replicaSet
! kubectl apply -f replicaset.yaml            # k apply -f replicaset.yaml

replicaset.apps/hostname unchanged


In [107]:
# 列印目前 replicaset 
! kubectl get replicaset

NAME       DESIRED   CURRENT   READY   AGE
hostname   3         3         3       15m


` kubectl get replicaset `
- `NAME` ReplicaSet 的名稱
- `DESIRED` 目標 Replica 數量
- `CURRENT` 目前 Replica 數量
- `READY` 當前 Ready的 Replica 數量
---


In [110]:
# 列印目前 rs(replicaset) 並且顯示 Labels
! kubectl get rs --show-labels

NAME       DESIRED   CURRENT   READY   AGE   LABELS
hostname   3         3         3       16m   app=hostname


In [111]:
# 列印目前 labels 有標註 tier:frontend 的 Pods
! kubectl get pods --selector=tier=frontend --show-labels -o wide

NAME             READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES   LABELS
hostname-5wdst   1/1     Running   0          16m   172.17.0.3   minikube   <none>           <none>            tier=frontend
hostname-74c7b   1/1     Running   0          16m   172.17.0.5   minikube   <none>           <none>            tier=frontend
hostname-dp5xn   1/1     Running   0          16m   172.17.0.4   minikube   <none>           <none>            tier=frontend


In [113]:
! kubectl get pods \
  --selector=tier=frontend \
  --no-headers \
  -o custom-columns=":status.podIP"

172.17.0.3
172.17.0.5
172.17.0.4


In [ ]:
# Ctrl+Shift+L
# minikube ssh
# ! curl 172.17.0.3:8080

curl: (7) Failed to connect to 172.17.0.3 port 8080 after 3062 ms: No route to host


In [ ]:
# 列印目前 Pods 和其Labels
! kubectl get pods --show-labels

NAME             READY   STATUS    RESTARTS   AGE    LABELS
hostname-tfndl   1/1     Running   0          3m4s   tier=frontend
hostname-x6bx8   1/1     Running   0          3m4s   tier=frontend
hostname-z57gd   1/1     Running   0          3m4s   tier=frontend


In [ ]:
! kubectl get pods \
  --selector=tier=frontend \
  --no-headers \
  -o custom-columns=":metadata.name"

hostname-tfndl
hostname-x6bx8
hostname-z57gd


In [ ]:
! kubectl delete pods --selector=tier=frontend 
! echo 
! kubectl get pods --selector=tier=frontend 

pod "hostname-tfndl" deleted
pod "hostname-x6bx8" deleted
pod "hostname-z57gd" deleted
^C

NAME             READY   STATUS        RESTARTS   AGE
hostname-82cwq   1/1     Running       0          11s
hostname-dffcp   1/1     Running       0          11s
hostname-tfndl   1/1     Terminating   0          3m22s
hostname-thpsr   1/1     Running       0          11s
hostname-x6bx8   1/1     Terminating   0          3m22s
hostname-z57gd   1/1     Terminating   0          3m22s


In [ ]:
! kubectl get pods --selector=tier=frontend 

NAME             READY   STATUS        RESTARTS   AGE
hostname-82cwq   1/1     Running       0          11s
hostname-dffcp   1/1     Running       0          11s
hostname-tfndl   1/1     Terminating   0          3m22s
hostname-thpsr   1/1     Running       0          11s
hostname-x6bx8   1/1     Terminating   0          3m22s
hostname-z57gd   1/1     Terminating   0          3m22s


In [ ]:
! kubectl scale --help

Set a new size for a deployment, replica set, replication controller, or
stateful set.

 Scale also allows users to specify one or more preconditions for the scale
action.

 If --current-replicas or --resource-version is specified, it is validated
before the scale is attempted, and it is guaranteed that the precondition holds
true when the scale is sent to the server.

Examples:
  # Scale a replica set named 'foo' to 3
  kubectl scale --replicas=3 rs/foo
  
  # Scale a resource identified by type and name specified in "foo.yaml" to 3
  kubectl scale --replicas=3 -f foo.yaml
  
  # If the deployment named mysql's current size is 2, scale mysql to 3
  kubectl scale --current-replicas=2 --replicas=3 deployment/mysql
  
  # Scale multiple replication controllers
  kubectl scale --replicas=5 rc/foo rc/bar rc/baz
  
  # Scale stateful set named 'web' to 3
  kubectl scale --replicas=3 statefulset/web

Options:
    --all=false:
	Select all resources in the namespace of the specified resource t

In [ ]:
! kubectl scale --replicas=5 replicaset hostname

replicaset.apps/hostname scaled


In [ ]:
! kubectl get pods --selector=tier=frontend 

NAME             READY   STATUS              RESTARTS   AGE
hostname-4fj9q   0/1     ContainerCreating   0          2s
hostname-6lf94   0/1     ContainerCreating   0          2s
hostname-82cwq   1/1     Running             0          2m43s
hostname-dffcp   1/1     Running             0          2m43s
hostname-thpsr   1/1     Running             0          2m43s


In [ ]:
! kubectl scale --replicas=4 replicaset hostname
! kubectl get pods --selector=tier=frontend 

replicaset.apps/hostname scaled
NAME             READY   STATUS        RESTARTS   AGE
hostname-4fj9q   1/1     Terminating   0          8s
hostname-6lf94   1/1     Running       0          8s
hostname-82cwq   1/1     Running       0          2m49s
hostname-dffcp   1/1     Running       0          2m49s
hostname-thpsr   1/1     Running       0          2m49s


In [ ]:
! kubectl scale --replicas=2 replicaset hostname
! kubectl get pods --selector=tier=frontend 

replicaset.apps/hostname scaled
NAME             READY   STATUS        RESTARTS   AGE
hostname-4fj9q   1/1     Terminating   0          25s
hostname-6lf94   1/1     Terminating   0          25s
hostname-82cwq   1/1     Running       0          3m6s
hostname-dffcp   1/1     Terminating   0          3m6s
hostname-thpsr   1/1     Running       0          3m6s


In [ ]:
! kubectl autoscale rs hostname --max=10 --min=2 --cpu-percent=50

horizontalpodautoscaler.autoscaling/hostname autoscaled


In [ ]:
! kubectl get pods --selector=tier=frontend 

NAME             READY   STATUS        RESTARTS   AGE
hostname-4fj9q   1/1     Terminating   0          38s
hostname-6lf94   1/1     Terminating   0          38s
hostname-82cwq   1/1     Running       0          3m19s
hostname-dffcp   1/1     Terminating   0          3m19s
hostname-thpsr   1/1     Running       0          3m19s


In [ ]:
# 刪除所有 replicaset 
! kubectl delete replicaset --all

replicaset.apps "frontend" deleted
replicaset.apps "hostname" deleted
replicaset.apps "hostname-deployment-6c545dc44c" deleted
